In [0]:
%run "../includes/configuration"

##Aggregate functions demo

####built in Aggregate functions

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
display(race_results_df)

In [0]:
demo_race_df = race_results_df.filter("race_year in (2019,2020)")
display(demo_race_df)

In [0]:
from pyspark.sql.functions import count, countDistinct, sum

In [0]:
demo_race_df.select(count("race_name")).show()
demo_race_df.select(countDistinct("race_name")).show()

In [0]:
demo_race_df.select(sum("points")).show()

In [0]:
demo_race_df.filter("driver_name= 'Lewis Hamilton'").select(sum("points"),countDistinct("race_name")) \
    .withColumnRenamed("sum(points)","total_points") \
    .withColumnRenamed("count(DISTINCT race_name)","number_of_wins").show()


In [0]:
demo_grouped_df=demo_race_df \
    .groupBy("race_year","driver_name") \
    .agg(sum("points").alias("total_points"),countDistinct("race_name").alias("number_of_wins")) \
    

In [0]:
display(demo_grouped_df)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank
driverRankSpec=Window.partitionBy("race_year").orderBy(desc("total_points"))
demo_grouped_df.withColumn("rank",rank().over(driverRankSpec)).show(100)